In [ ]:
import os
import cv2 as cv
from matplotlib import pyplot as plt
import numpy as np
import tables, pickle
import math

In [ ]:
def store(folder, data):
    with open(folder,'wb') as f:
        pickle.dump(data,f)

In [ ]:
folder='data'


In [ ]:
def compute(video,frames):
    # Source for this code: Geeks for Geeks
# The video feed is read in as
# a VideoCapture object
    i=0
    field_magnitude=np.zeros((frames,128,128))
    field_angle=np.zeros((frames,128,128))
    cap = cv.VideoCapture(video)
  
# ret = a boolean return value from
# getting the frame, first_frame = the
# first frame in the entire video sequence
    ret, first_frame = cap.read()
  
# Converts frame to grayscale because we
# only need the luminance channel for
# detecting edges - less computationally 
# expensive
    prev_gray = cv.cvtColor(first_frame, cv.COLOR_BGR2GRAY)
  
# Creates an image filled with zero
# intensities with the same dimensions 
# as the frame
    mask = np.zeros_like(first_frame)
    fps=15
    size=[128,128] 
# Sets image saturation to maximum
    mask[..., 1] = 255
    image=None
    orig=None
    fl=None
    try:  
        while(cap.isOpened()):
      
    # ret = a boolean return value from getting
    # the frame, frame = the current frame being
    # projected in the video
            ret, frame = cap.read()
      
    # Opens a new window and displays the input
    # frame
            #cv.imshow("input", frame)
      
    # Converts each frame to grayscale - we previously 
    # only converted the first frame to grayscale
            gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
      
    # Calculates dense optical flow by Farneback method
            flow = cv.calcOpticalFlowFarneback(prev_gray, gray, 
                                       None,
                                       0.5, 3, 15, 3, 5, 1.2, 0)
      
    # Computes the magnitude and angle of the 2D vectors
            magnitude, angle = cv.cartToPolar(flow[..., 0], flow[..., 1])
            fl=magnitude, angle*180/np.pi/2
       
            
            #temp1=val1.resize(128**2,1)
            #temp2=val2.resize(128**2,1)
            field_magnitude[i]=fl[0]
            field_angle[i]=fl[1]

    # Sets image hue according to the optical flow 
    # direction
            mask[..., 0] = angle * 180 / np.pi / 2
      
    # Sets image value according to the optical flow
    # magnitude (normalized)
            mask[..., 2] = cv.normalize(magnitude, None, 0, 255, cv.NORM_MINMAX)
       
    # Converts HSV to RGB (BGR) color representation
            rgb = cv.cvtColor(mask, cv.COLOR_HSV2BGR)
       
    # Opens a new window and displays the output frame
       #cv.imshow("dense optical flow", rgb)
            image=rgb
            orig=gray
       #compute()
       #show()
    # Updates previous frame
            prev_gray = gray
            i=i+1
    # Frames are read by intervals of 1 millisecond. The
    # programs breaks out of the while loop when the
    # user presses the 'q' key
            if cv.waitKey(1) & 0xFF == ord('q'):
                break
    except:
        print("Video has ended.")
# The following frees up resources and
# closes all windows
    cap.release()
    cv.destroyAllWindows()
    return field_magnitude, field_angle

In [ ]:
def video(file,vid, frame):
    size = 128, 128
    duration = 10
    fps = 15
    name=vid
    output=name[0:6] + '.mp4'
    out = cv.VideoWriter(output, cv.VideoWriter_fourcc(*'mp4v'), fps, (size[1], size[0]), False)
    f=tables.open_file(file)
    data = f.get_node('/rawStims')[:] # load activity
    f.close()
    for i in range(frame):
        d = data[i]
        out.write(d)
    out.release()
    

In [ ]:
def response(file):
    f=tables.open_file(file)
    f.list_nodes # Show all variables available
#print(f)
    data = f.get_node('/psths')[:] # load activity
    f.close()
    return data[0]

In [ ]:
for file in os.listdir(folder):
    #f=open(os.path.join(folder,file),'r')
    f=str(folder)+'/'+str(file)
    
    spikes=response(f)

   
    name=str(file)

    video(f,name,len(spikes))

    newfolder=name[0:6]

    if not os.path.isdir(newfolder):
        os.mkdir(newfolder)

    stimulus=newfolder+'.mp4'
    field=compute(stimulus,len(spikes))
    f1=newfolder+'/'+newfolder+'-magnitude'
    store(f1,field[0])
    f2=newfolder+'/'+newfolder+'-angle'
    store(f2,field[1])
    del field
    f3=newfolder+'/'+newfolder+'-responses'
    
    
    
    store(f3,spikes)
    del spikes
    
    